In [1]:
import EOVoxelCraft as eovc
import geopandas as gpd
from pathlib import Path

c:\Users\hoeh_pa\AppData\Local\miniconda3\envs\sits\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
crafter = eovc.init('cdse')
credentials_path = Path(r"C:\Users\hoeh_pa\Desktop\EOVoxelCraft\credentials\cdse_credentials.json")
crafter.set_credentials(credentials_path)

In [3]:
collections = crafter.retrieve_collections()
print(collections)

['COP-DEM', 'S2GLC', 'TERRAAQUA', 'SENTINEL-3', 'SENTINEL-5P', 'SENTINEL-1-RTC', 'SENTINEL-1', 'SMOS', 'LANDSAT-7', 'CCM', 'LANDSAT-5', 'LANDSAT-8', 'ENVISAT', 'SENTINEL-6', 'GLOBAL-MOSAICS', 'SENTINEL-2']


In [4]:
gdf = gpd.read_file(r"C:\Users\hoeh_pa\Desktop\EOVoxelCraft\demo_files\data\TUM_OTN.shp")
gdf['geometry'] = gdf['geometry']
arguments = dict(
    shp=gdf, 
    bands=['B02', 'B03', 'B04'],
    resolution=20,
    collection='SENTINEL-2', 
    start_date='2021-06-01', 
    end_date='2021-07-10', 
    download_folder=r'C:\Users\hoeh_pa\Desktop\EOVoxelCraft\demo_files\data\downloads', 
    num_workers=8
    )

In [5]:
items = crafter.search(**arguments)

In [9]:
items[0].id

'S2A_MSIL2A_20210602T101031_N0500_R022_T32UPU_20230313T040411.SAFE'

In [ ]:
selected_items = []
for item in items:
    if item.properties["tileId"] == "32UPU" and item.properties['processingLevel'] == "S2MSIL2A" and item.properties['cloudCover'] < 10.0:
        selected_items.append(item)
print(f"Selected items: {len(selected_items)}")

In [ ]:
import odc.stac 
bounds = list(crafter.get_param('shp', raise_error=True).bounds.values[0])
crs = crafter.get_param('shp', raise_error=True).crs
data = odc.stac.load(
    items, 
    crs=crs.to_epsg(), 
    resolution=20,
    )